# Notebook to plot persistence from a dataframe produced by Pers_in_visit


In [ ]:
from astropy.io import fits
import glob, os, shutil, pickle, bz2, gc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import sigmaclip
from scipy.optimize import curve_fit
from scipy.special import gammaincc, gamma
from astropy.wcs import WCS
from astropy.stats import histogram
from itertools import product
from multiprocessing import Pool

%matplotlib notebook

In [ ]:
pwd

In [ ]:
# The project dir 
pdir = '/user/gennaro/Functional_work/WFC3_persistence/py_progs/short_term_persistence/'

#The mosaic dir
mdir = pdir+'/Mosaic_hi_res_folder/'

#The dir to save/load the Persistence curves dataframes
sdir = pdir+'/PD_dataframes_dir/'

In [ ]:
#Single and double exponential models to be fitted to the data

def decay1(t,a1,t1):
    e1 = a1*np.exp(-t/t1)
    return e1

def intdec1(t,a1,t1):
    tu = t[1:]
    td = t[:-1]
    k  = -a1*t1
    return k*(np.exp(-tu/t1)-np.exp(-td/t1))/(tu-td)
    
def decay2(t,a1,t1,a2,t2):
    e1 = a1*np.exp(-t/t1)
    e2 = a2*np.exp(-t/t2)
    return e1+e2

def intdec2(t,a1,t1,a2,t2):
    tu = t[1:]
    td = t[:-1]
    k1,k2  = -a1*t1, - a2*t2
    
    return k1*(np.exp(-tu/t1)-np.exp(-td/t1))/(tu-td) + k2*(np.exp(-tu/t2)-np.exp(-td/t2))/(tu-td)

#Single exponential models plus a constant

def intdec1_plusconst(t,a1,t1,q):
    tu = t[1:]
    td = t[:-1]
    k  = -a1*t1
    return k*(np.exp(-tu/t1)-np.exp(-td/t1))/(tu-td) +q

def dec1_plusconst(t,a1,t1,q):
    e1 = a1*np.exp(-t/t1)
    return e1+q


#Shifted power law model

def shpwl(t,t0,A,index):
    return A * ((t+t0)/1000)**index

def intshpwl(t,t0,A,index):
    tu = t[1:]
    td = t[:-1]

    if (index == -1.):
        return A*np.log( (tu+t0)/(td+t0) )
    else:
        return A/(1+index) * ( ((tu+t0)/1000)**(1+index) - ((td+t0)/1000)**(1+index) )/(tu-td)
    
    
#Schechter like model

def schechter(t,phi,alpha,tstar):
    x = t/tstar
    return phi*(x**alpha)*np.exp(-x)

def intschechter(t,phi,alpha,tstar):
    x = t/tstar

    tu = x[1:]
    td = x[:-1]

    g1 = gammaincc(alpha+1,td)
    g2 = gammaincc(alpha+1,tu)
    
    diff = gamma(alpha+1)*(g1-g2)
    
    return phi*diff


#Geometric median calculation function

from scipy.spatial.distance import cdist, euclidean

def geometric_median(X, eps=1e-5):
    y = np.mean(X, 0)

    while True:
        D = cdist(X, [y])
        nonzeros = (D != 0)[:, 0]

        Dinv = 1 / D[nonzeros]
        Dinvs = np.sum(Dinv)
        W = Dinv / Dinvs
        T = np.sum(W * X[nonzeros], 0)

        num_zeros = len(X) - np.sum(nonzeros)
        if num_zeros == 0:
            y1 = T
        elif num_zeros == len(X):
            return y
        else:
            R = (T - y) * Dinvs
            r = np.linalg.norm(R)
            rinv = 0 if r == 0 else num_zeros/r
            y1 = max(0, 1-rinv)*T + min(1, rinv)*y

        if euclidean(y, y1) < eps:
            return y1, D

        y = y1



In [ ]:
#Cell to restore the hf5 store with the dataframe for each visit
#df = pd.DataFrame()

#for vis in ['01','02','03']:

 #   with bz2.BZ2File(sdir+'Test_DF_visit_'+vis+'.pbz2', 'r') as f:
 #       dfp = pickle.load(f)
        #dfp['Visit'] = vis
        #df = df.append(dfp, ignore_index=True)
    

In [ ]:
print('Total number of entries',len(df))
df.head()

In [ ]:
gBMs    =   [np.ones(len(df),dtype=np.bool_)]
gBMs.append( (df['Ind_pers'] != 7) )
gBMs.append( (df['Ind_pers'] != 7) & (df['Read index'] != 1) & (df['Ind_stim'] != 6) )
gBMs.append( (df['Ind_pers'] == 7) & (df['deltat'] > 10 ))
gBMs.append( (df['Ind_pers'] > 7)  & (df['Read index'] != 1) )
gBMs.append( (df['Ind_pers'] < 7)  & (df['Read index'] != 1) )
gBMs.append( (df['Ind_pers'] != 7) & (df['Read index'] != 1) )

fig = plt.figure(figsize=(15,4*len(gBMs)))
ax = []

for i,gBM in enumerate(gBMs):

    df2 = df[gBM]

    ax.append(fig.add_subplot(len(gBMs),2,1+i*2))
    ax.append(fig.add_subplot(len(gBMs),2,2+i*2))
    
    col = np.where(df2['Ind_pers']>=7,np.repeat('#55adff',len(df2)),np.repeat('#12679B',len(df2)))

    ax[-2].scatter(df2['tfromstim'],df2['meancurr']-df2['background'],s=2,alpha=0.7,c=col)
    ax[-1].scatter(df2['tfromstim'],df2['meancurr']-df2['background'],s=2,alpha=0.7,c=col)

    step = 25.
    tmin = np.min(df2['tfromstim'].values)
    tmax = np.max(df2['tfromstim'].values)

    tmed   = []
    medsig = []

    while tmin <= tmax:

        BM =  (df2['tfromstim'].values > tmin-step/2.) & (df2['tfromstim'].values <= (tmin+ step/2.))
        tmin += step
        if (np.sum(BM) > 5):
            medsig.append(np.nanmedian(df2['meancurr'][BM].values-df2['background'][BM].values))
            tmed.append(np.median(df2['tfromstim'][BM].values))
    
            #gv = sigmaclip((df2['meancurr'][BM].values-df2['background'][BM].values),2.5,2.5)[0]
            #if (len(gv) >= 3):
                #medsig.append(np.nanmedian(gv))
                #tmed.append(np.median(df2['tfromstim'][BM].values))
    
    ax[-2].plot(tmed,medsig,c='r')
    ax[-1].plot(tmed,medsig,c='r')               
    ax[-1].plot(np.array([1,10000]),700*np.array([1.,1/10000.]),c='black')

    ax[-1].set_xscale('log')
    ax[-1].set_yscale('log')
    ax[-1].set_ylim(0.001,100)
    ax[-2].set_ylim(-10,100)
    ax[-1].set_xlim(1,10000)
    ax[-2].set_xlim(1,10000)

plt.tight_layout()

In [ ]:
gstims = np.unique(df['Ind_stim'] )

fig = plt.figure(figsize=(15,4*gstims.size))

ax = []

for j,i in enumerate(gstims):
    gBMh = (df['Ind_stim'] == i) & (df['Ind_pers'] != 7) & (df['Read index'] != 1) &  (df['Ind_pers'] <7)
    
    if (np.sum(gBMh) > 0):
        df2 = df[gBMh]
        ax.append(fig.add_subplot(gstims.size,2,j*2+1))
        ax.append(fig.add_subplot(gstims.size,2,j*2+2))

        col = np.where(df2['Ind_pers']>=7,np.repeat('#55adff',len(df2)),np.repeat('#12679B',len(df2)))
        
        ax[-2].scatter(df2['tfromstim'],df2['meancurr']-df2['background'],s=2,alpha=0.7,c=col)
        ax[-1].scatter(df2['tfromstim'],df2['meancurr']-df2['background'],s=2,alpha=0.7,c=col)

        tmedh = []
        medsigh= []
        
        step = 25
        tmin = np.min(df2['tfromstim'].values)
        tmax = np.max(df2['tfromstim'].values)

        while tmin <= tmax:
    
            BM =  (df2['tfromstim'].values > tmin-step/2.) & (df2['tfromstim'].values <= (tmin+ step/2.))
            tmin += step
            if (np.sum(BM) > 5):
                tmedh.append(np.median(df2[BM]['tfromstim']))
                #medsigh.append(np.nanmedian(sigmaclip((df2[BM]['meancurr'].values-df2[BM]['background'].values),2.5,2.5)[0]))
                medsigh.append(np.nanmedian(df2[BM]['meancurr'].values-df2[BM]['background']))
    
        ax[-2].plot(tmed,medsig,c='r')
        ax[-1].plot(tmed,medsig,c='r')
        ax[-2].plot(tmedh,medsigh,c='orange')
        ax[-1].plot(tmedh,medsigh,c='orange')
    else:
        ax[-2].plot(tmed,medsig,c='r')
        ax[-1].plot(tmed,medsig,c='r')
 
    ax[-1].plot(np.array([1,10000]),700*np.array([1,1/10000.]),c='black')
    
    ax[-1].set_xscale('log')
    ax[-1].set_yscale('log')
    ax[-1].set_ylim(0.001,100)
    ax[-2].set_ylim(-10,100)
    ax[-1].set_xlim(1,10000)
    ax[-2].set_xlim(1,10000)

plt.tight_layout()

In [ ]:
gstims = np.unique(df['Ind_stim'] )

fig = plt.figure(figsize=(15,4*gstims.size))

ax = []

for j,i in enumerate(gstims):
    gBMh = (df['Ind_stim'] == i) & (df['Ind_pers'] != 7) & (df['Read index'] != 1) &  (df['Ind_pers'] >7)
    
    if (np.sum(gBMh) > 0):
        df2 = df[gBMh]
        ax.append(fig.add_subplot(gstims.size,2,j*2+1))
        ax.append(fig.add_subplot(gstims.size,2,j*2+2))

        col = np.where(df2['Ind_pers']>=7,np.repeat('#55adff',len(df2)),np.repeat('#12679B',len(df2)))
        
        ax[-2].scatter(df2['tfromstim'],df2['meancurr']-df2['background'],s=2,alpha=0.7,c=col)
        ax[-1].scatter(df2['tfromstim'],df2['meancurr']-df2['background'],s=2,alpha=0.7,c=col)

        tmedh = []
        medsigh= []
        
        step = 25
        tmin = np.min(df2['tfromstim'].values)
        tmax = np.max(df2['tfromstim'].values)

        while tmin <= tmax:
    
            BM =  (df2['tfromstim'].values > tmin-step/2.) & (df2['tfromstim'].values <= (tmin+ step/2.))
            tmin += step
            if (np.sum(BM) > 5):
                tmedh.append(np.median(df2[BM]['tfromstim']))
                #medsigh.append(np.nanmedian(sigmaclip((df2[BM]['meancurr'].values-df2[BM]['background'].values),2.5,2.5)[0]))
                medsigh.append(np.nanmedian(df2[BM]['meancurr'].values-df2[BM]['background']))
    
        ax[-2].plot(tmed,medsig,c='r')
        ax[-1].plot(tmed,medsig,c='r')
        ax[-2].plot(tmedh,medsigh,c='orange')
        ax[-1].plot(tmedh,medsigh,c='orange')
    else:
        ax[-2].plot(tmed,medsig,c='r')
        ax[-1].plot(tmed,medsig,c='r')
  
    ax[-1].plot(np.array([1,10000]),700*np.array([1,1/10000.]),c='black')
    ax[-1].set_xscale('log')
    ax[-1].set_yscale('log')
    ax[-1].set_ylim(0.001,100)
    ax[-2].set_ylim(-10,100)
    ax[-1].set_xlim(1,10000)
    ax[-2].set_xlim(1,10000)

plt.tight_layout()

In [ ]:
mf=1.2
levels = lev_d * np.array([1,np.power(mf,1),np.power(mf,2),np.power(mf,3),np.power(mf,4),np.power(mf,5),np.power(mf,6),np.power(mf,7)])

fig = plt.figure(figsize=(15,4*levels.size))

ax = []

colors = np.array(['#e41a1c', '#377eb8', '#4daf4a','#a44a7c','#123321','#cad111','#111cad','#abccdb','#dd88a1','#ab0011',
                   '#aa11ca','#b18522','#097023','#909090','#090909','#FDEEDF','DDDDDD','#556611','#0911AA','#8FD45A'])

for j,(low,high) in enumerate(zip(list(levels[:-1]),list(levels[1:]))):
    gBMh = (df['Ind_pers'] != 7) & (df['Read index'] != 1) & (df['Stim'] >= low) & (df['Stim']<high)

    ax.append(fig.add_subplot(levels.size,2,j*2+1))
    ax.append(fig.add_subplot(levels.size,2,j*2+2))

    if (np.sum(gBMh) > 0):
        df2 = df[gBMh]

        ax[-2].set_title('Low:'+str(low)+' High:'+str(high)+' Np:'+str(np.sum(gBMh)))
        ax[-1].set_title('Low:'+str(low)+' High:'+str(high)+' Np:'+str(np.sum(gBMh)))
        
        ax[-2].scatter(df2['tfromstim'],df2['meancurr']-df2['background'],s=2,alpha=0.7,c=[df2['Ind_stim'].values])
        ax[-1].scatter(df2['tfromstim'],df2['meancurr']-df2['background'],s=2,alpha=0.7,c=[df2['Ind_stim'].values])

        tmedh = []
        medsigh= []
        
        step = 50
        tmin = np.min(df2['tfromstim'].values)
        tmax = np.max(df2['tfromstim'].values)

        while tmin <= tmax:
    
            BM =  (df2['tfromstim'].values > tmin-step/2.) & (df2['tfromstim'].values <= (tmin+ step/2.))
            tmin += step
            if (np.sum(BM) > 5):
                tmedh.append(np.median(df2[BM]['tfromstim']))
                #medsigh.append(np.nanmedian(sigmaclip((df2[BM]['meancurr'].values-df2[BM]['background'].values),2.5,2.5)[0]))
                medsigh.append(np.nanmedian(df2[BM]['meancurr'].values-df2[BM]['background']))
    
        ax[-2].plot(tmed,medsig,c='r')
        ax[-1].plot(tmed,medsig,c='r')
        ax[-2].plot(tmedh,medsigh,c='orange')
        ax[-1].plot(tmedh,medsigh,c='orange')
    else:
        ax[-2].plot(tmed,medsig,c='r')
        ax[-1].plot(tmed,medsig,c='r')
  
    ax[-1].plot(np.array([1,10000]),700*np.array([1,1/10000.]),c='black')
    ax[-1].set_xscale('log')
    ax[-1].set_yscale('log')
    ax[-1].set_ylim(0.001,100)
    ax[-2].set_ylim(-10,100)
    ax[-1].set_xlim(1,10000)
    ax[-2].set_xlim(1,10000)

plt.tight_layout()